# QVERIFY: Verification Deep Dive

This notebook explores the verification engine of QVERIFY.

## Topics Covered
1. NeuralSilVer verifier architecture
2. Verification condition generation
3. SMT solving for quantum programs
4. Counterexample analysis
5. LLM-assisted lemma synthesis

In [ ]:
from qverify import QVerify, QuantumProgram, Specification
from qverify.verification import NeuralSilVer, create_z3_interface
from qverify.verification.vc_generator import VCGenerator, generate_vcs
from qverify.verification.smt_interface import create_solver, Z3Solver
from qverify.verification.counterexample import CounterexampleAnalyzer

## 1. NeuralSilVer Architecture

NeuralSilVer combines SMT-based verification with LLM-assisted lemma synthesis.

In [ ]:
# Initialize components
smt_solver = create_solver("z3")  # or "mock" for testing

verifier = NeuralSilVer(
    smt_solver=smt_solver,
    llm=None,  # Can add LLM for lemma hints
    timeout=30.0,
    enable_lemma_hints=True
)

print("NeuralSilVer verifier initialized")
print(f"  SMT Solver: {type(smt_solver).__name__}")
print(f"  Timeout: {verifier.timeout}s")

## 2. Verification Condition Generation

VCs are generated from the program and specification using quantum weakest precondition calculus.

In [ ]:
from qverify.core.specification import Precondition, Postcondition

# Create a simple program
program = QuantumProgram.from_silq("""
def hadamard(q: qubit) {
    q = H(q);
    return q;
}
""")

# Create specification
spec = Specification(
    precondition=Precondition.from_string("in_basis(q, |0⟩)"),
    postcondition=Postcondition.from_string("superposition(q)"),
    name="hadamard_spec"
)

# Generate VCs
vc_gen = VCGenerator()
vcs = vc_gen.generate(program, spec)

print(f"Generated {len(vcs)} verification conditions:")
for vc in vcs:
    print(f"  - {vc.name}")

## 3. SMT Solving

Each VC is checked using an SMT solver (Z3).

In [ ]:
from qverify.verification.smt_interface import SolverResult

# Check a simple formula
test_formula = "(assert (=> (in_basis q zero) (superposition q)))"

# Using the solver
result = smt_solver.check(test_formula, timeout=10.0)

print(f"Formula: {test_formula}")
print(f"Result: {result.status.name}")
print(f"Time: {result.time_seconds:.3f}s")

## 4. Full Verification Example

In [ ]:
# Run full verification
result = verifier.verify(program, spec)

print(f"\nVerification Result:")
print(f"  Status: {result.status.name}")
print(f"  Time: {result.time_seconds:.2f}s")
print(f"  Verified VCs: {result.verified_conditions}")

if result.solver_stats:
    print(f"\nSolver Statistics:")
    for key, value in result.solver_stats.items():
        print(f"  {key}: {value}")

## 5. Counterexample Analysis

When verification fails, counterexamples help diagnose the issue.

In [ ]:
analyzer = CounterexampleAnalyzer()

print("Counterexample analyzer can:")
print("  - Diagnose violation types (precondition, postcondition, invariant)")
print("  - Extract concrete quantum states from SMT models")
print("  - Suggest specification fixes")

## 6. Quantum-Specific SMT Theory

QVERIFY uses a custom SMT theory for quantum states.

In [ ]:
from qverify.verification.smt_interface import QUANTUM_SMT_PRELUDE

print("Quantum SMT Theory (T_Q):")
print("="*50)
print(QUANTUM_SMT_PRELUDE[:500])
print("...")

## Summary

The QVERIFY verification pipeline:

1. **Specification** → Precondition, Postcondition, Invariants
2. **VC Generation** → Hoare triples, loop VCs, frame conditions
3. **SMT Solving** → Z3 with quantum theory T_Q
4. **Result** → Valid, Invalid (+ counterexample), or Unknown
5. **LLM Hints** → Lemma suggestions for stuck VCs